# LC Distance from Branch: Data Prep

**Last Updated**: 3 August

1. Load & Merge Data
2. Fix Wrong/Outlier Distances (includes API call with R)
3. Bucket Distances
4. Export to CSV

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  

## 1. Load & Merge Data

- During SQL pull, kept only observations with model tier not null
- All mailings start in **Jan 2017** for all states except **Texas**, which starts in July.
- All mailings in this dataset end in **June 2018**

In [4]:
data = pd.read_csv("./Data/sql_allchecks_2017_notnull_modeltier.csv",
                  header = None,
                  names = ['cashed', 'casheddollars', 'issuedate', 'amountfinanced', 
                          'branchid', 'contrid', 'namesid', 'newacct', 'newcontr', 'account',
                          'city', 'state', 'zipcode', 'grade',
                          'beacon', 'ownorrent', 'singleorjoint', 'modeltier'], 
                   low_memory = False)

data.cashed = data.cashed.fillna(0)
data.casheddollars = data.casheddollars.fillna(0)
data.ownorrent = data.ownorrent.str.upper()
data.grade = data.grade.str.upper()

data = data.assign(beaconbucket = pd.cut(data.beacon, 
                                    bins = [-1,300, 600,650,700,750,800,851],
                                    labels = ['Beacon < 300',
                                             'Beacon 300-599',
                                             'Beacon 600-649',
                                             'Beacon 650-699',
                                             'Beacon 700-749',
                                             'Beacon 750-799',
                                             'Beacon 800-850']))

# data.groupby('state').issuedate.min()
# data.groupby('state').issuedate.max()

In [5]:
# Load distance from branch data from RF database
distdata = pd.read_csv("./Data/sqleverything_distancefromBR.csv", 
                       header = None,
                       names = ['state', 'branchid', 'zipcode', 'city', 'distbr'])
distdata.zipcode = pd.to_numeric(distdata.zipcode, errors="coerce")

# Attach to main data
full = pd.merge(data, distdata, how = "left", on = ['state', 'branchid', 'city', 'zipcode'])
full.shape

(17645848, 20)

**`trunc` dataframe: keep only observations WITH distance information**

In [6]:
trunc = full[pd.notnull(full.distbr)].copy()

# Number of missing observations
full.shape[0] - trunc.shape[0]

3115758

## 2. Fix Distances

### A) Outliers 
**Distance entered wrongly** for branch 116 with zipcode `37814`. Distance was recorded as over 2000 miles. Distance between branch and residential zipcode `37722` = **29.42** miles. Replace this value in the data.

In [7]:
# trunc.loc[trunc.distbr > 2000][['branchid', 'zipcode']].drop_duplicates()
trunc.loc[trunc['distbr'] > 2000, 'distbr'] = 29.42

### B) False 0 distances
- In general, most of the NON-ZERO distances from branch seem accurate (checked by hand).
- However, many zipcodes have '0' for distance when they are not (We should only have 0 distance when the customer and branch are in the same zipcode)
- Here I fix the 0s in the smaller dataset `trunc` for our analysis, rather than the overall dataframe since it'll be too many addresses for a free API call.

**Prepare csv of all zipcodes to run API call with**:

In [11]:
branches = pd.read_csv("./Data/branch_details_short.csv", header = None,
                      names = ['branchid', 'brst', 'brzip'])

# Convert zipcodes to numeric to match other dataset
branches.brzip = pd.to_numeric(branches.brzip.str.slice(0, 5), errors = 'coerce')

# Remove those with no branch zipcode
# branches.loc[pd.isnull(branches.brzip)]
branches = branches.dropna()

# Select all those with distance = 0
# Remove observations where the branch zipcode is same as the target zipcode (will definitely be 0)
tofix = pd.merge(
    trunc.loc[(trunc.distbr == 0)][['branchid', 'zipcode', 'distbr']].drop_duplicates(),
    branches[['branchid', 'brzip']],
    how = 'left',
    on = 'branchid')
tofix = tofix.loc[tofix.zipcode != tofix.brzip]

##################################
# Export to CSV to call APIs in R
##################################

#tofix.to_csv("./Data/zipcodes_tofix.csv")

**Run an API Call to find new distances**

- `zip_codes_tofix.csv` was fed into R to do the API calls through Zip-codes.com 
- See `fix_distances_apicall.R` for documentation
- Resulting csv was `fixed_dist_19june.csv`

**Replace new distfrombr values back into `trunc` table**

- Add new values in as `newdist`
- Rename old distbr column as `originalbr`
- Add both together (since the originals would have been 0 anyway) to form the modified `distbr`

In [20]:
# Load pulled data
fixeddist = pd.read_csv("./Data/fixed_dist_19june.csv")

# Keep relevant columns & rename
fixeddist = fixeddist[['branchid', 
                       'zipcode', 
                       'distbr']].rename(columns = {'distbr': 'newdist'})

# Left join new distances
# Fill in 0 for all the accurate fields we didn't API call for
trunc = pd.merge(trunc, 
                 fixeddist, 
                 how = 'left', 
                 on = ['branchid', 'zipcode'])
trunc.newdist = trunc.newdist.fillna(0)

# Rename old column to 'original br'
trunc.rename(columns = {'distbr': 'originalbr'}, inplace = True)

# Create new 'distbr' column based on sum of original and new distances 
trunc = trunc.assign(distbr = trunc.originalbr + trunc.newdist)
trunc.head()

,cashed,casheddollars,issuedate,amountfinanced,branchid,contrid,namesid,newacct,newcontr,account,...,zipcode,grade,beacon,ownorrent,singleorjoint,modeltier,beaconbucket,originalbr,newdist,distbr
0,1.0,5055.95,2018-03-30 00:00:00.000,5055.95,103,NaN,NaN,NaN,NaN,NaN,...,39759,A,678,O,J,0.5,Beacon 650-699,0.00,0.0,0.00
1,1.0,1422.97,2018-04-30 00:00:00.000,1422.97,75,NaN,NaN,NaN,NaN,NaN,...,70128,B,622,O,S,2.0,Beacon 600-649,17.49,0.0,17.49
2,1.0,2345.69,2018-03-30 00:00:00.000,2345.69,1,34303.0,7188.0,8173,63788.0,NaN,...,70816,C,595,R,S,2.0,Beacon 300-599,0.00,0.0,0.00
3,1.0,1422.97,2018-04-30 00:00:00.000,1422.97,1,34385.0,7253.0,8236,63870.0,NaN,...,70802,B,619,R,S,2.0,Beacon 600-649,9.00,0.0,9.00
4,1.0,1306.47,2018-04-30 00:00:00.000,1306.47,6,55006.0,19484.0,78341,79985.0,NaN,...,39501,C,595,R,S,2.0,Beacon 300-599,4.48,0.0,4.48


## 3. Bucket Distances

In [21]:
trunc = trunc.assign(distbucket = pd.cut(trunc.distbr, 
                                    bins = [-1,5,10,15,20,25,30,60],
                                    labels = [5,10,15,20,25,30,60]))
trunc.groupby('distbucket').size().reset_index()

,distbucket,0
0,5,3998066
1,10,4629121
2,15,2939302
3,20,1459436
4,25,748160
5,30,568647
6,60,187271


## 4. Export dataframe for future use
**Last exported**: 28 June

In [ ]:
#trunc.to_csv("./Data.rf_2017_modeltiers_lcs_trunc_28june.csv")